# Imports

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
from djimaging.user.alpha.schemas.alpha_schema import *
from djimaging.user.alpha.utils import populate_alpha
from djimaging.utils.dj_utils import get_primary_key

populate_alpha.load_alpha_config(schema_name=populate_alpha.SCHEMA_PREFIX + "ca")
populate_alpha.load_alpha_schema(create_schema=False, create_tables=False)

In [ ]:
schema

In [ ]:
plt.hist(Field().fetch('pixel_size_um'))

In [ ]:
(Presentation & "stim_name='gChirp'").fetch('pixel_size_um').min(), (Presentation & "stim_name='gChirp'").fetch('pixel_size_um').max()

# GLMs

## Preprocessing

In [ ]:
populate_alpha.populate_rf_glms_traces(verbose=True, processes=5)

In [ ]:
GLMDNoiseTraceParams()

In [ ]:
GLMDNoiseTrace().plot1(xlim=(100, 101))

### Fit

In [ ]:
RFGLMParams().add_default(**{
    'rf_glm_params_id': 10,
    'filter_dur_s_past': 1.2,
    'filter_dur_s_future': 0.2,
    'df_ts': (10,),
    'df_ws': (9,),
    'betas': (0.005,),
    'kfold': 0,
    'metric': 'mse',
    'output_nonlinearity': 'none',
    'other_params_dict': {
        'frac_test': 0,
        'min_iters': 100,
        'max_iters': 2000,
        'step_size': 0.1,
        'tolerance': 5,
        'alphas': (1.0,),
        'verbose': 100,
        'n_perm': 20,
        'min_cc': 0.2,
        'seed': 42,
        'fit_R': False,
        'fit_intercept': True,
        'init_method': 'random',
        'atol': 1e-05,
        'distr': 'gaussian',
        'step_size_finetune': 0.03}
}, skip_duplicates=True)
RFGLMParams()

In [ ]:
from tqdm.notebook import tqdm
from datajoint.errors import LostConnectionError
import time

for key in tqdm((RFGLM().key_source - RFGLM().proj()).fetch('KEY')):
    rf_entry = RFGLM()._fetch_and_compute(key=key, clear_outputs=True, suppress_outputs=True)
    try:
        RFGLM().insert1(rf_entry, allow_direct_insert=True)
    except LostConnectionError:
        dj.conn()
        time.sleep(3)
        RFGLM().insert1(rf_entry, allow_direct_insert=True)

In [ ]:
(RFGLM() & RFGLM().fetch('KEY')[0]).fetch1()['model_dict']

#### Plot fits

In [ ]:
RFGLM & (RoiKind & "roi_kind='soma'")

In [ ]:
key = get_primary_key(RFGLM & (RoiKind & "roi_kind='soma'"))
key.pop('rf_glm_params_id')
for rf_glm_params_id in RFGLMParams.fetch('rf_glm_params_id'):
    print(rf_glm_params_id)
    try:
        (RFGLM & key & dict(rf_glm_params_id=rf_glm_params_id)).plot1()
        plt.show()
    except:
        print('\tno data')

## Metrics

In [ ]:
populate_alpha.populate_rf_glm_properties(verbose=True, processes=10)

## Contours

In [ ]:
GLMContoursParams().add_default(rf_contours_params_id=1, blur_std=1, blur_npix=2, norm_kind='amp_one', levels=(0.25, 0.3, 0.35), skip_duplicates=True)
GLMContoursParams()

In [ ]:
GLMContours().populate("rf_glm_params_id=10", make_kwargs=dict(plot=False), processes=10, display_progress=True)

In [ ]:
(GLMContours() & "rf_cdia_um>150" & (SplitRFGLM() & "split_qidx>0.4")).plot1()

In [ ]:
(GLMContours() & "largest_contour_ratio>0.8" & "largest_contour_ratio<0.9" & "rf_glm_params_id=10" & (SplitRFGLM() & "split_qidx>0.4")).plot1()

In [ ]:
(GLMContours() & "largest_contour_ratio>0.9" & "largest_contour_ratio<1.0" & "rf_glm_params_id=10" & (SplitRFGLM() & "split_qidx>0.4")).plot1()

In [ ]:
plt.hist((GLMContours & "rf_glm_params_id=10").fetch('irregular_index'));

### Metrics

In [ ]:
GLMContoursParams()

In [ ]:
GLMContourMetrics().populate(make_kwargs=dict(plot=False), processes=10, order='random')

In [ ]:
(GLMContourMetrics() & (GLMContours() & "rf_cdia_um>150") & "rf_glm_params_id=10" & (SplitRFGLM() & "split_qidx>0.4")).plot1()

In [ ]:
GLMContourOffset().populate(display_progress=True, processes=1)

In [ ]:
for key in (RFGLM() & "rf_glm_params_id=10" & (GLMContours & "largest_contour_ratio<1")).fetch('KEY')[:3]:
    (GLMContours() & key).plot1()

In [ ]:
for key in (RFGLM() & "rf_glm_params_id=10" & (CellTags & "cell_tag='d5'") & (RoiKind & "roi_kind='soma'")).fetch('KEY'):
    (GLMContours() & key).plot1()

In [ ]:
SplitRFGLM() & (RoiKind & "roi_kind='field'")

## Plots

In [ ]:
key = get_primary_key(RFGLM)
key.pop('rf_glm_params_id')
for rf_glm_params_id in RFGLMParams.fetch('rf_glm_params_id'):
    print(rf_glm_params_id)
    (SplitRFGLM & dict(rf_glm_params_id=rf_glm_params_id) & (RoiKind & "roi_kind='roi'")).plot(sort=True)